## Running Spark in the Cloud
- Connecting to Google Cloud Storage
    - Upload the parquet data in the `data` directory to GCS. Check the [`cloud.md`](cloud.md) for how to do that
- [Connecting Spark jobs to GCS](https://youtu.be/Yyz293hBVcQ)
- [Connecting Spark to BQ](https://youtu.be/HIm2BOj8C0Q)
- [Setting Up Dataproc Cluster](https://www.youtube.com/watch?v=osAiAYahvh8&list=PPSV)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = '/home/aayomide/.google/credentials/google_credentials.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/07/02 05:22:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
# read data from the google cloud storage
df_green = spark.read.parquet('gs://data_lake_de-zmcmp-spark-nytaxi/pq/green/*/*')

In [6]:
df_green.count()

1009792